# Memory Representation of Values

- https://dev.realworldocaml.org/runtime-memory-layout.html
- https://ocamlverse.github.io/content/runtime.html#ocaml-runtime
- https://github.com/ocaml/ocaml/blob/trunk/runtime/caml/mlvalues.h

Ocaml uses a uniform memory representation to store a value <br/>

Ocaml value is a single memory word that is either immediate integer (the lowest bit is nonzero) or a pointer (the lowest bit is zero) <br/>

A pointer may point to a block storing an ocaml value on th heap managed by OCaml runtime or a block storing C values <br/>

Every block managed by OCaml runtime has a header (32 bits on 32-bit machine and 64-bits on 64-bit machine) and followed by variable-length data that's either opaque bytes or an array of fields. GC never inspects opaque bytes. GC inspects only OCaml fields. <br/>

**32-bit header**

| block size | color | tag |
| --- | --- | --- |
| 22 bits | 2 bits | 8 bits |

On 32-bit machine, Array and String has limitation 2^22 = 16 MB. For bigger string, `Bigarray` should be used.

**64-bit header**

| block size | color | tag |
| --- | --- | --- |
| 54 bits | 2 bits | 8 bits |

**Color field** is used by GC to keep track of its state during mark-and-sweep collection<br/>

**Tag field** is used to tell the runtime system and the GC about the kind
of data contained in the block.
Tag values 0 through 245 are used for things such as declared types.
For example, in the type

```ocaml
type t = A | B of int | C of float
``` 
The tag for `A` will be 0, for `B` will be 1, and for `C` will be 2. OCaml therefore allows for a maximum number of 246 options in a sum type. Any more, and compilation will fail.

The tags 246-255 are reserved for:

| Tag | Usage | Define in Obj|
| --- | --- |---| 
| 246 | Lazy values | lazy_tag|
| 247 | Closures |closure_tag|
| 248 | Objects |object_tag|
| 249 | Infix values inside closures | infix_tag|
| 250 | Forward values (lazy values that have been forced) | forward_tag|
| 251 | Abstract values (shouldn't be scanned by GC) | no_scan_tag/abstract_tag|
| 252 | Strings | string_tag|
| 253 | Floating-point value | double_tag|
| 254 | Flat floating-point array | double_array_tag|
| 255 | Custom tag (for bundling non-OCaml data) | custom_tag|

**Note: Any data with tag >= 251 is not scanned by the GC.**

- int or char are stored directly as a value, shifted left by 1 bit, with the least significant bit set to 1.

- unit, [], false are all stored as OCaml int 0.

- true is stored as OCaml int 1.

- Foo | Bar variants are stored as ascending OCaml ints, starting from 0.

- Foo | Bar of int variants with parameters are boxed, while variants with no parameters are unboxed.

- Polymorphic variants with parameters are boxed with an extra header word to store the value, as compared to normal variants. Polymorphic variants with no parameters are unboxed.

- Floating-point numbers are stored as a block with a single field containing the double-precision float.

- Strings are word-aligned byte arrays with an explicit length.

- [1; 2; 3] lists are stored as 1::2::3::[] where [] is an int, and h::t a block with tag 0 and two parameters.

- Tuples, records, and arrays are stored as a C array of values. Arrays can be variable size, but tuples and records are fixed-size.

- Records or arrays that are all float use a special tag for unboxed arrays of floats, or records that only have float fields.

In [82]:
(* 
    int, char, unit, [], true, false, variant constant constructor are stored as integer
*)

Obj.is_block (Obj.repr 1) ;;    (* false *)
Obj.is_block (Obj.repr 'a') ;;  (* false *)
Obj.is_block (Obj.repr () ) ;;  (* false *)
Obj.is_block (Obj.repr true) ;; (* false *)
Obj.is_block (Obj.repr false) ;;(* false *)
Obj.is_block (Obj.repr `A) ;;   (* false *)
type on_off = On | Off ;;       
Obj.is_block (Obj.repr On) ;;   (* false *)

- : bool = false


- : bool = false


- : bool = false


- : bool = false


- : bool = false


- : bool = false


type on_off = On | Off


- : bool = false


In [83]:
(*
    The followings are boxed.
    Variants or Polymorphic variants with paramaters,
    Floating-point numbers (stored as a block with a single field containing the number)
    Objects
    First-class module 
*)

type 'a list = Nil | Cons of 'a * 'a list ;;

Obj.is_block (Obj.repr Nil) ;; (* false *)
Obj.is_block (Obj.repr (Cons (1,Nil))) ;; (* true *)

Obj.is_block (Obj.repr (`A 10)) ;; (* true *)
Obj.is_block (Obj.repr (object val x = 10 end)) ;; (* true *)

module type T = sig val x : int end ;;
module M : T = struct let x = 10 end ;;

let packed = (module M : T) ;;
Obj.is_block (Obj.repr packed) ;; (* true *)
Obj.tag (Obj.repr packed) ;; (* 0 *)

type 'a list = Nil | Cons of 'a * 'a list


- : bool = false


- : bool = true


- : bool = true


- : bool = true


module type T = sig val x : int end


module M : T


val packed : (module T) = <module>


- : bool = true


- : int = 0


In [88]:
(*
    tuples, records, arrays are boxed with tag 0.
    OCaml GC doesn't track them cause they are C values.
*)

Obj.is_block (Obj.repr (1,2)) ;; (* true *)
Obj.is_block (Obj.repr [|1; 2|]) ;; (* true *)
type human = { name : string ; age : float } ;;
Obj.is_block (Obj.repr ({ name = "Ryan" ; age = 6.5 } ));; (* true *)

- : bool = true


- : bool = true


type human = { name : string; age : float; }


- : bool = true


In [144]:
(*
    Array or Records containing only float types are sotred in block tag double_array_tag.
*)
let fa = [| 11.1; 22.2 |] ;;
type fr = { x : float ; y : float } ;;
let p = { x = 11.1; y = 22.2 } ;;

Obj.is_block (Obj.repr fa) ;; (* true *)
Obj.tag (Obj.repr fa) = Obj.double_array_tag ;;  (* true *)

Obj.double_field (Obj.repr fa) 0 = fa.(0) ;;

Obj.is_block (Obj.repr p) ;;  (* true *)
Obj.tag (Obj.repr p) = Obj.double_array_tag ;;  (* true *)
Obj.double_field (Obj.repr p) 0 ;;
Obj.double_field (Obj.repr p) 1 ;;

val fa : float array = [|11.1; 22.2|]


type fr = { x : float; y : float; }


val p : fr = {x = 11.1; y = 22.2}


- : bool = true


- : bool = true


- : bool = true


- : bool = true


- : bool = true


- : float = 11.1


- : float = 22.2


In [1]:
(* variant types without paramaters are stored as integer, starting with 0 *)
type abc = A | B | C ;;
((Obj.magic (Obj.repr A)) : int) ;;
((Obj.magic (Obj.repr B)) : int) ;;
((Obj.magic (Obj.repr C)) : int) ;;

(*
    variant type with paramaters are boxed with value tag ascending from 0.
    paramaters are stored as words in the block.
*)
type axis = X of float | Y of float | Z of float ;;
Obj.is_block (Obj.repr (X 1.1)) ;; (* true *) 
Obj.tag (Obj.repr (X 1.1)) ;; (* 0 *)
Obj.tag (Obj.repr (Y 1.1)) ;; (* 1 *)
Obj.tag (Obj.repr (Z 1.1)) ;; (* 2 *) 

(Obj.magic (Obj.field (Obj.repr (X 10.2)) 0) : float ) ;;
(Obj.magic (Obj.field (Obj.repr (X 5.5)) 0) : float ) ;;

type abc = A | B | C


- : int = 0


- : int = 1


- : int = 2


type axis = X of float | Y of float | Z of float


...

- : int = 0


- : int = 1


- : int = 2


- : float = 10.2


- : float = 5.5


Polymorphic variants use more memory space than normal variants when parameters are included in the data type constructors. Normal variants use the tag byte to encode the variant value and save the fields for the contents, but this single byte is insufficient to encode the hashed value for polymorphic variants. They must allocate a new block (with tag 0) and store the value in there instead. Polymorphic variants with constructors thus use one word of memory more than normal variant constructors.

Another inefficiency over normal variants is when a polymorphic variant constructor has more than one parameter. Normal variants hold parameters as a single flat block with multiple fields for each entry, but polymorphic variants must adopt a more flexible uniform memory representation, since they may be reused in a different context across compilation units. They allocate a tuple block for the parameters that is pointed to from the argument field of the variant. There are thus three additional words for such variants, along with an extra memory indirection due to the tuple.

The extra space usage is generally not significant in a typical application, and polymorphic variants offer a great deal more flexibility than normal variants. However, if you’re writing code that demands high performance or must run within tight memory bounds, the runtime layout is at least very predictable. The OCaml compiler never switches memory representation due to optimization passes. This lets you predict the precise runtime layout by referring to these guidelines and your source code.
<br/>-- Real World OCaml

In [12]:
(*
    polymorphic variant without paramaters is stored as unboxed integer. 
    The integer value is the result of the hasing of the name of the variant.
*)

#load "ocamlcommon.cma" ;;
#require "ocaml-compiler-libs.common";; 
Btype.hash_variant "A";;
(Obj.magic (Obj.repr `A) : int) ;;

- : int = 65


- : int = 65
